<h2 align='left' style='color:blue'>Dropout Regularization In Deep Neural Network</h2>

- This is a dataset that describes sonar chirp (noise) returns bouncing off different services. 
- The 60 input variables are the strength of the returns at different angles. 
- It is a **binary classification problem** that requires a model to differentiate rocks from metal cylinders.

- Dataset information: 
https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)
- Download it from here: 
https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#df = pd.read_csv("./sonar_dataset.csv")
# We don't have column name. Remove the column names with header = None.
df = pd.read_csv("./sonar_dataset.csv", header=None)   
df.sample(5)

0       1       2       3       4       5       6       7       8   \
145  0.0721  0.1574  0.1112  0.1085  0.0666  0.1800  0.1108  0.2794  0.1408   
169  0.0130  0.0120  0.0436  0.0624  0.0428  0.0349  0.0384  0.0446  0.1318   
61   0.0135  0.0045  0.0051  0.0289  0.0561  0.0929  0.1031  0.0883  0.1596   
17   0.0192  0.0607  0.0378  0.0774  0.1388  0.0809  0.0568  0.0219  0.1037   
88   0.0274  0.0242  0.0621  0.0560  0.1129  0.0973  0.1823  0.1745  0.1440   

         9   ...      51      52      53      54      55      56      57  \
145  0.0795  ...  0.0309  0.0127  0.0082  0.0281  0.0117  0.0092  0.0147   
169  0.1375  ...  0.0084  0.0100  0.0018  0.0035  0.0058  0.0011  0.0009   
61   0.1908  ...  0.0037  0.0084  0.0102  0.0096  0.0024  0.0037  0.0028   
17   0.1186  ...  0.0331  0.0131  0.0120  0.0108  0.0024  0.0045  0.0037   
88   0.1808  ...  0.0113  0.0108  0.0085  0.0047  0.0074  0.0104  0.0161   

         58      59  60  
145  0.0157  0.0129   M  
169  0.0033  0.0026   M  
61   0.0030  0.0030   R  
17   0.0112  0.0075   R  
88   0.0220  0.0173   R  

[5 rows x 61 columns]

In [5]:
df.shape

(208, 61)

In [6]:
# check for nan values
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [7]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [8]:
df[60].value_counts() # label is not skewed

M    111
R     97
Name: 60, dtype: int64

In [9]:
# X = df.drop(60, axis=1)
X = df.drop(60, axis="columns") # we drop columns 60.
y = df[60]
print('y.head():')
print(y.head())
X.head()

y.head():
0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object


0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      50      51      52      53      54      55      56  \
0  0.2111  ...  0.0232  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180   
1  0.2872  ...  0.0125  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140   
2  0.6194  ...  0.0033  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316   
3  0.1264  ...  0.0241  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050   
4  0.4459  ...  0.0156  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072   

       57      58      59  
0  0.0084  0.0090  0.0032  
1  0.0049  0.0052  0.0044  
2  0.0164  0.0095  0.0078  
3  0.0044  0.0040  0.0117  
4  0.0048  0.0107  0.0094  

[5 rows x 60 columns]

In [10]:
y = pd.get_dummies(y, drop_first=True) 
# after the dummpy is generate, 
# We drop the orginal variable R (Rock) or M (Mines) and keep the 1 and 0.
y.sample(5) # R --> 1 and M --> 0

R
175  0
18   1
155  0
30   1
146  0

In [11]:
y.value_counts()

R
0    111
1     97
dtype: int64

In [12]:
X.head()

0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      50      51      52      53      54      55      56  \
0  0.2111  ...  0.0232  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180   
1  0.2872  ...  0.0125  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140   
2  0.6194  ...  0.0033  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316   
3  0.1264  ...  0.0241  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050   
4  0.4459  ...  0.0156  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072   

       57      58      59  
0  0.0084  0.0090  0.0032  
1  0.0049  0.0052  0.0044  
2  0.0164  0.0095  0.0078  
3  0.0044  0.0040  0.0117  
4  0.0048  0.0107  0.0094  

[5 rows x 60 columns]

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [14]:
print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_test.shape:', X_test.shape)
print('y_test.shape:', y_test.shape)

X_train.shape: (156, 60)
y_train.shape: (156, 1)
X_test.shape: (52, 60)
y_test.shape: (52, 1)


In [15]:
X_train.head()

0       1       2       3       4       5       6       7       8   \
67   0.0368  0.0403  0.0317  0.0293  0.0820  0.1342  0.1161  0.0663  0.0155   
14   0.0124  0.0433  0.0604  0.0449  0.0597  0.0355  0.0531  0.0343  0.1052   
164  0.0163  0.0198  0.0202  0.0386  0.0752  0.1444  0.1487  0.1484  0.2442   
179  0.0394  0.0420  0.0446  0.0551  0.0597  0.1416  0.0956  0.0802  0.1618   
19   0.0126  0.0149  0.0641  0.1732  0.2565  0.2559  0.2947  0.4110  0.4983   

         9   ...      50      51      52      53      54      55      56  \
67   0.0506  ...  0.0058  0.0091  0.0160  0.0160  0.0081  0.0070  0.0135   
14   0.2120  ...  0.0078  0.0083  0.0057  0.0174  0.0188  0.0054  0.0114   
164  0.2822  ...  0.0027  0.0077  0.0026  0.0031  0.0083  0.0020  0.0084   
179  0.2558  ...  0.0118  0.0146  0.0040  0.0114  0.0032  0.0062  0.0101   
19   0.5920  ...  0.0153  0.0092  0.0035  0.0098  0.0121  0.0006  0.0181   

         57      58      59  
67   0.0067  0.0078  0.0068  
14   0.0196  0.0147  0.0062  
164  0.0108  0.0083  0.0033  
179  0.0068  0.0053  0.0087  
19   0.0094  0.0116  0.0063  

[5 rows x 60 columns]

# Using Deep Learning Model

## Model without Dropout Layer

In [16]:
import tensorflow as tf
from tensorflow import keras

2024-04-18 21:09:37.761111: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 21:09:37.761141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 21:09:37.761920: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 21:09:37.767164: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 21:09:38.554667: W tensorflow/compiler/tf2

In [17]:
model = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100


2024-04-18 21:09:39.112083: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-18 21:09:39.142945: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-18 21:09:39.143157: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

20/20 [==============================] - 1s 2ms/step - loss: 0.6953 - accuracy: 0.5000
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6628 - accuracy: 0.6154
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6346 - accuracy: 0.6667
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6212 - accuracy: 0.6923
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5841 - accuracy: 0.7564
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5446 - accuracy: 0.7628
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4975 - accuracy: 0.7756
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4724 - accuracy: 0.8077
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4245 - accuracy: 0.8141
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4021 - accuracy: 0.8526
Epoch 11/100
20/20 [=

In [18]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 1.2338 - accuracy: 0.7500


[1.233786940574646, 0.75]

Training Accuracy >>> Test Accuracy

In [19]:
y_pred = model.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 2ms/step
[6.6361252e-08 9.6479803e-01 9.8548734e-01 3.5074343e-06 9.9999630e-01
 9.9994802e-01 4.6464419e-03 9.9999833e-01 9.6162930e-06 9.9999750e-01]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [20]:
y_test[:10]

R
186  0
155  0
165  0
200  0
58   1
34   1
151  0
18   1
202  0
62   1

In [21]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.85      0.78        27
           1       0.80      0.64      0.71        25

    accuracy                           0.75        52
   macro avg       0.76      0.75      0.75        52
weighted avg       0.76      0.75      0.75        52



### Model with Dropout Layer

In [22]:
modeld = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

modeld.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modeld.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 2ms/step - loss: 0.7494 - accuracy: 0.4615
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6757 - accuracy: 0.5641
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7044 - accuracy: 0.4872
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7077 - accuracy: 0.4808
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6868 - accuracy: 0.5256
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6904 - accuracy: 0.5192
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6763 - accuracy: 0.5705
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6662 - accuracy: 0.5641
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6600 - accuracy: 0.5641
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6396 - accuracy: 0.6282
Epoch 11/

In [23]:
modeld.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.9393 - accuracy: 0.7308


[0.9392620921134949, 0.7307692170143127]

Training Accuracy is still good but Test Accuracy Improved

In [24]:
y_pred = modeld.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 2ms/step
[5.6673775e-07 9.4118482e-01 9.8759371e-01 4.2402344e-06 9.9998844e-01
 9.9245423e-01 3.4179077e-01 9.9999058e-01 2.9082277e-05 9.9999535e-01]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [25]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.89      0.77        27
           1       0.82      0.56      0.67        25

    accuracy                           0.73        52
   macro avg       0.75      0.72      0.72        52
weighted avg       0.75      0.73      0.72        52



**You can see that by using dropout layer test accuracy increased from 0.77 to 0.82**